In [1]:
import tensorflow as tf
import random
import numpy as np
import cv2
import glob
from tqdm import tqdm
from PIL import Image
from pathlib import Path
import pickle
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Activation, Dropout, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import TensorBoard
import time

In [2]:
training_data = []
DATADIR = "C:/Users/User/Desktop/ML/Clothes"
CATEGORIES = ["Notags", "Tags"]
IMG_SIZE = 256
def create_training_data():
    for category in CATEGORIES:  # notags and tags categories

        path = os.path.join(DATADIR,category)  # create path to notags and tags
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=nptag 1=tag

        for img in tqdm(os.listdir(path)):  # iterate over each image per notags and tags
            try:
                img_array = cv2.imread(os.path.join(path,img))  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize image size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # pass errors
                pass

create_training_data() 

print(len(training_data))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1659/1659 [00:56<00:00, 29.27it/s]

5678


In [3]:
import random
#Shuffle data
random.shuffle(training_data)

In [4]:
X = []
y = []
#Split to features and labels
for features,label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y)    

In [5]:
#Normalize RGB from (0,255) to (0,1)
X = X/255.0

In [7]:
#Store features array on disk
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()
#Store labels array on disk
pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [ ]:
#Load features and labels without a need to extract and clean data
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)
pickle_in.close()
pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)
pickle_in.close()

In [8]:
#Name for tensorboard logs AlexNEt implementation 70-73% valaccuracy
NAME = "Tags-notags-clothes-CNN-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(256,256,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
#Name for tensorboard logs My CNN 69% valaccuracy
NAME = "Tags-notags-clothes-CNN-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
# Model Architecture 
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.35))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(1))
model.add(Activation('sigmoid'))



In [9]:
model.compile(loss="binary_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 62, 62, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 30, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 384)       8

In [10]:
epochs = 10
model.fit(X, y, batch_size=32, validation_split=0.2, epochs=epochs, callbacks=[tensorboard])

Epoch 1/10
142/142 [==============================] - 21s 76ms/step - loss: 10.3273 - accuracy: 0.6208 - val_loss: 0.6487 - val_accuracy: 0.7315
Epoch 2/10
142/142 [==============================] - 5s 34ms/step - loss: 0.6882 - accuracy: 0.6654 - val_loss: 0.5730 - val_accuracy: 0.7262
Epoch 3/10
142/142 [==============================] - 5s 33ms/step - loss: 0.6460 - accuracy: 0.6716 - val_loss: 0.7420 - val_accuracy: 0.6778
Epoch 4/10
142/142 [==============================] - 5s 33ms/step - loss: 0.5944 - accuracy: 0.7094 - val_loss: 0.6116 - val_accuracy: 0.7350
Epoch 5/10
142/142 [==============================] - 5s 32ms/step - loss: 0.5872 - accuracy: 0.7061 - val_loss: 0.6108 - val_accuracy: 0.7157
Epoch 6/10
142/142 [==============================] - 5s 32ms/step - loss: 0.6872 - accuracy: 0.6920 - val_loss: 0.8709 - val_accuracy: 0.7298
Epoch 7/10
142/142 [==============================] - 5s 33ms/step - loss: 0.6187 - accuracy: 0.7030 - val_loss: 0.6914 - val_accuracy: 0.59